In [1]:
import numpy as np
import tensorflow as tf
import codecs
import unidecode
import re

from random import shuffle
from nltk.corpus import stopwords

In [2]:
TRAIN_SIZE = 102582 + 49000 + 32519 #+ 1899
TEST_SIZE = 1000

NUM_FEATURES = 50
SEQ_LEN = 250
CELL_SIZE = 128
BATCH_SIZE = 64
NUM_CLASESS = 3  # negative positive neutral

HIDDEN_LAYER_SIZE = 32

print(TRAIN_SIZE+TEST_SIZE)

185101


In [3]:
wordsList = np.load('wordsList.npy')
print('cargar el diccionario!')
wordsList = wordsList.tolist() #Originally loaded as numpy array
wordsList = [word.decode('UTF-8') for word in wordsList] #Encode words as UTF-8
wordVectors = np.load('wordVectors.npy')
print ('cargar word vectors!')
print(len(wordsList))
print(wordVectors.shape)

cargar el diccionario!
cargar word vectors!
400000
(400000, 50)


In [4]:
# cargar los reviews and labels
ids = np.load('idsMatrix.npy')
labels_d = np.load('labels.npy')

ids = np.concatenate((ids, np.load('idsMatrix_3_7.npy')), axis = 0)
labels_d = np.concatenate((labels_d, np.load('labels_3_7.npy')), axis=0)

print(ids.shape)
print(labels_d.shape)

(185101, 250)
(185101,)


In [5]:
# shuffle trainning examples
num_examples = labels_d.shape[0]
indexs = [i for i in range(num_examples)]
shuffle(indexs)
ids = ids[indexs]
labels_d = labels_d[indexs]

In [6]:
#ids[:,0:SEQ_LEN].shape
#ids = ids[:,0:SEQ_LEN]

In [7]:
index_t = 0
def getTrainBatch():
    global index_t
    if index_t >= TRAIN_SIZE: index_t = 0
    labels_ = []
    arr = np.zeros([BATCH_SIZE, SEQ_LEN])
    for i in range(BATCH_SIZE):
        if labels_d[index_t+i] == 0: 
            labels_.append([1,0,0])
        if labels_d[index_t+i] == 1:
            labels_.append([0,1,0])
        if labels_d[index_t+i] == 2:
            labels_.append([0,0,1])
    arr = ids[index_t : index_t + BATCH_SIZE]
    index_t += BATCH_SIZE
    #print index_t
    return arr, labels_

# DATA es de 152000, nuestro test son los ultimos 5000 reviews
def getTestBatch(n):
    labels_ = []
    arr = np.zeros([n, SEQ_LEN])
    for i in range(n):
        if labels_d[i+TRAIN_SIZE] == 0:
            labels_.append([1,0,0]) # negative
        if labels_d[i+TRAIN_SIZE] == 1:
            labels_.append([0,1,0]) # positive
        if labels_d[i+TRAIN_SIZE] == 2:
            labels_.append([0,0,1]) #neutral
    arr = ids[TRAIN_SIZE:TRAIN_SIZE + n]
    return arr, labels_

In [8]:
sess = tf.Session()
tf.reset_default_graph()

labels = tf.placeholder(dtype = tf.float32, shape = [None, NUM_CLASESS]) # batch x classes 
input_data = tf.placeholder(dtype = tf.int32, shape = [None, SEQ_LEN]) # batch  x seq_len 

hin_f = tf.placeholder(dtype = tf.float32, shape = [None, CELL_SIZE]) # batch x cell
hin_b = tf.placeholder(dtype = tf.float32, shape = [None, CELL_SIZE]) # batch x cell


In [9]:
def my_attention(inputs, hidden_layer_size):
    X = tf.reshape(inputs, [-1, 2 * CELL_SIZE])
    Y = tf.layers.dense(X, hidden_layer_size, activation=tf.nn.relu)
    logits = tf.layers.dense(Y, 1, activation= None)
    logits = tf.reshape(logits, [-1, SEQ_LEN, 1])
    alphas = tf.nn.softmax(logits, dim=1)
    encoded_sentence = tf.reduce_sum(inputs * alphas, axis=1)
    
    return encoded_sentence, alphas

In [10]:
#data = tf.Variable(tf.zeros([None, SEQ_LEN, NUM_FEATURES]),dtype=tf.float32)
data = tf.nn.embedding_lookup(wordVectors,input_data)

# creamos bidirectional gru
# layer 0
rnn_fw_cell_0 = tf.contrib.rnn.GRUCell(CELL_SIZE)
rnn_bw_cell_0 = tf.contrib.rnn.GRUCell(CELL_SIZE)
# layer 1
rnn_fw_cell_1 = tf.contrib.rnn.GRUCell(CELL_SIZE)
rnn_bw_cell_1 = tf.contrib.rnn.GRUCell(CELL_SIZE)

list_rnn_fw = [rnn_fw_cell_0, rnn_fw_cell_1]
list_rnn_bw = [rnn_bw_cell_0, rnn_bw_cell_1]

#outputs, _  = tf.nn.bidirectional_dynamic_rnn(cell_bw=rnn_bw_cell,cell_fw=rnn_fw_cell,inputs = data,dtype = tf.float32)
                                                #initial_state_bw=hin_b, initial_state_fw=hin_b)
outputs,_,_ = tf.contrib.rnn.stack_bidirectional_dynamic_rnn(cells_bw = list_rnn_fw, cells_fw = list_rnn_bw,
                                                             inputs = data, dtype=tf.float32)

#outputs = tf.concat(outputs, axis=2)

encoded, alphas = my_attention(outputs, HIDDEN_LAYER_SIZE)

logits = tf.layers.dense(encoded, 3, activation=None)
prediction = tf.argmax(logits, 1)

true_false = tf.equal(prediction, tf.argmax(labels,1))
accuracy = tf.reduce_mean(tf.cast(true_false, tf.float32))

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits))
#loss = tf.losses.softmax_cross_entropy(onehot_labels=labels, logits=logits)
optimizer = tf.train.AdamOptimizer().minimize(loss)

Instructions for updating:
dim is deprecated, use axis instead
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [11]:
################################################ test###############################################

In [12]:
def filter_text(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    return text

# reviews es una lista de reviews y polarities una lista de int
def get_format_data(reviews, polarities):
    arr = np.zeros((len(polarities), SEQ_LEN),dtype='uint32')
    arr_labels = []
    for r,review in enumerate(reviews):
        review = filter_text(unidecode.unidecode(review))
        review = review.split()
        for w, word in enumerate(review):
            if w >= SEQ_LEN: break
            try:
                arr[r][w] = wordsList.index(word)
            except ValueError:
                arr[r][w] = 399999
    for l in polarities:
        if l == 0: arr_labels.append([1,0,0])
        if l == 1: arr_labels.append([0,1,0])
        if l == 2: arr_labels.append([0,0,1])
    return arr, arr_labels
filename1 = 'test/intertass-test-2017.csv'
text1 = codecs.open(filename1, encoding='utf-8')
next(text1)

sentences1 = []
polarity1 = []

i = 0
for line in text1:
    lista = line.split(',') # asumimos que el review no contiene comas 
    w = str(lista[1]).split('\n')[0]
    if w == 'NONE' or w == 'NEU': polarity1.append(2) # neutro
    if w == 'P': polarity1.append(1) # positive
    if w == 'N': polarity1.append(0)   # negative
    sentence = lista[0]
    #sentence = filter_text(unidecode.unidecode(sentence))
    sentences1.append(sentence)
    i += 1
print i

NUM_EJEM_1 = len(sentences1)
at, bt = get_format_data(sentences1, polarity1)

1899


In [13]:
####################################################################################################

In [ ]:
import datetime

tf.summary.scalar('Loss', loss)
tf.summary.scalar('Accuracy_Train', accuracy)
merged = tf.summary.merge_all()
validation_summ = tf.summary.scalar('Accuracy_Test', accuracy)
logdir = "tensorboard/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "/"
#writer = tf.summary.FileWriter(logdir, sess.graph)

In [1]:
sess = tf.InteractiveSession()
saver = tf.train.Saver()
sess.run(tf.global_variables_initializer())

writer = tf.summary.FileWriter(logdir, sess.graph)

# nuestros placeholder no tiene definido el batch_size, aqui indicamos con
#cuantos reviews trabajaremos
# nuestro getTrai_Batch trabaja con 32 reviews

#inh_f = np.zeros([BATCH_SIZE,CELL_SIZE])
#inh_b = np.zeros([BATCH_SIZE,CELL_SIZE])
#inh_f_v = np.zeros([128, CELL_SIZE])
#inh_b_v = np.zeros([128, CELL_SIZE])

i = 0
for epoch in range(50):    # NUM_EPOCHS
    for _ in range(int(TRAIN_SIZE/BATCH_SIZE)): # TRAIN_SIZE / BATCH_SIZE = steps for 1 epoch
       #Next Batch of reviews
        nextBatch, nextBatchLabels = getTrainBatch();
        sess.run(optimizer, {input_data: nextBatch, labels: nextBatchLabels})
                            #hin_f: inh_f, hin_b: inh_b})
   
        #Write summary to Tensorboard
        if (i % 50 == 0):
            summary = sess.run(merged, {input_data: nextBatch, labels: nextBatchLabels})
                                       #hin_f: inh_f, hin_b: inh_b})
            writer.add_summary(summary, i)
            a ,b = getTestBatch(128)
            summary2 = sess.run(validation_summ, {input_data: a, labels: b})
                                   #hin_f: inh_f_v, hin_b: inh_b_v})
            writer.add_summary(summary2, i)

        #Save the network every 10,000 training iterations
        if (i % 1000 == 0 and i != 0):
            save_path = saver.save(sess, "models/model_GRU_Attention.ckpt", global_step=i)
            print("saved to %s" % save_path)
            
        i += 1
    print("epoch", epoch)
writer.close()

NameError: name 'tf' is not defined

In [ ]:
# cargamos un modelo ya pre-entrenado
sess = tf.InteractiveSession()
saver = tf.train.Saver()
saver.restore(sess, tf.train.latest_checkpoint('models'))

In [103]:
# probando la red con 5000 ejemplos 
a0, b0 =getTestBatch(TEST_SIZE)
p = sess.run(accuracy, {input_data: a0, labels: b0})
print (p*100)

91.07999801635742


In [ ]:
# probar con ejemplos del TASS

In [12]:
def filter_text(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    return text

In [13]:
# reviews es una lista de reviews y polarities una lista de int
def get_format_data(reviews, polarities):
    arr = np.zeros((len(polarities), SEQ_LEN),dtype='uint32')
    arr_labels = []
    for r,review in enumerate(reviews):
        review = filter_text(unidecode.unidecode(review))
        review = review.split()
        for w, word in enumerate(review):
            if w >= SEQ_LEN: break
            try:
                arr[r][w] = wordsList.index(word)
            except ValueError:
                arr[r][w] = 399999
    for l in polarities:
        if l == 0: arr_labels.append([1,0,0])
        if l == 1: arr_labels.append([0,1,0])
        if l == 2: arr_labels.append([0,0,1])
    return arr, arr_labels

In [14]:
filename1 = 'test/intertass-test-2017.csv'
text1 = codecs.open(filename1, encoding='utf-8')
next(text1)

sentences1 = []
polarity1 = []

i = 0
for line in text1:
    lista = line.split(',') # asumimos que el review no contiene comas 
    w = str(lista[1]).split('\n')[0]
    if w == 'NONE' or w == 'NEU': polarity1.append(2) # neutro
    if w == 'P': polarity1.append(1) # positive
    if w == 'N': polarity1.append(0)   # negative
    sentence = lista[0]
    #sentence = filter_text(unidecode.unidecode(sentence))
    sentences1.append(sentence)
    i += 1
print i

NUM_EJEM_1 = len(sentences1)

1899


In [15]:
# tenemos que convertir los reviews a una matriz con idexs y los labels a one hot encoding

In [16]:
a, b = get_format_data(sentences1, polarity1)

In [17]:
p = sess.run(accuracy, {input_data: a, labels: b})
print (p*100)

72.30120897293091


In [105]:
a[0]

array([399999,    110,  13045, 201534,     58,    147,     14,     61,
           81,     33, 201534,   5942,    117,      7,  17448,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      